**Sudoku Convex Optimization Solver**  

Before we get started, the code will assume familiarity with the following:  
1) The complete set of rules in [Sudoku](https://en.wikipedia.org/wiki/Sudoku)  
2) General formatting and modeling approach for [Linear Programming](https://en.wikipedia.org/wiki/Linear_programming)






Below, we'll install PuLP, an open source linear programming solver as a matrix framework for stating our problem

In [22]:
!pip install pulp

     |████████████████████████████████| 40.6MB 100kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for amply: filename=amply-0.1.2-cp36-none-any.whl size=16572 sha256=802e874b4d28af671dd4995a7b3b5ea3e7ecc58506df0abafa41316760bd65dc
  Stored in directory: /root/.cache/pip/wheels/84/18/f7/e5c3ed13ed5bb721763f77d4a924331d59ef115ce61c9d26eb
Successfully built amply


We'll install necessary packages, using a combination of NumPy and Pandas to display the input puzzle into a dataframe

In [23]:
import pulp
import numpy as np
import pandas as pd

Now we'll pass the sudoku puzzle as a 2D array.    
For values we don't know, we'll populate the corresponding cell with a 0.    
For values we know, we'll populate the corresponding cell with the value.  
Then we'll convert it to a df with labels for our row, columns to check that the initial sudoku is correct.

In [24]:
puzzle = [[0, 0, 5, 0, 2, 0, 0, 0, 0], \
          [3, 9, 0, 1, 8, 6, 0, 0, 0], \
          [0, 0, 8, 4, 3, 0, 1, 0, 0], \
          [1, 3, 4, 0, 0, 0, 0, 0, 7], \
          [0, 2, 0, 6, 7, 4, 0, 3, 0], \
          [6, 0, 0, 0, 0, 0, 4, 9, 2], \
          [0, 0, 6, 0, 4, 2, 9, 0, 0], \
          [0, 0, 0, 9, 5, 1, 0, 7, 8], \
          [0, 0, 0, 0, 6, 0, 2, 0, 0]] 

df = pd.DataFrame(puzzle, index = np.arange(9) + 1, columns = np.arange(9) + 1)
df.index.name, df.columns.name = 'Row #', 'Column #'
df

Column #,1,2,3,4,5,6,7,8,9
Row #,,,,,,,,,
1,0,0,5,0,2,0,0,0,0
2,3,9,0,1,8,6,0,0,0
3,0,0,8,4,3,0,1,0,0
4,1,3,4,0,0,0,0,0,7
5,0,2,0,6,7,4,0,3,0
6,6,0,0,0,0,0,4,9,2
7,0,0,6,0,4,2,9,0,0
8,0,0,0,9,5,1,0,7,8
9,0,0,0,0,6,0,2,0,0


Now we'll start stating the rules of sudoku as a linear program.

Typically, creating a Linear Program involves a 4 step approach:  
1) Stating whether we are creating a minimization or maximization problem  
2) Defining variables and their range of possible values  
3) Writing our problem constraints (as a linear combination of our variables)  
4) Describe the minimization/maximization we are aiming for (as a linear combination of our variables)

**First Step (Minimization or Maximization):**  
In the scope of this problem, it won't matter whether we chose a minimization or maximization objective.  
Here, we are simply looking for the inputs that will solve all our constraints.  
Below, we'll instantiate the linear program and arbitrarily define it as a maximization problem

In [25]:
problem = pulp.LpProblem('Problem', pulp.LpMaximize) #defining a maximization linear program with name 'Problem'

**Second Step (Variable Definition):**  
We'll define a 3D array of binary variables, $X_{ijk}$, to tell whether a number/digit occurs within a specified row and column  
There's some math below needed to model our constraints, so we'll use some shorthand:  
- $i$ will indicate digits 1 through 9  
- $j$ will indicate rows 1 through 9
- $k$ will indicate columns 1 through 9
  
So $X_{321}$ will indicate whether the value 3 occurs in row 2, column 1




In [26]:
nums = range(1, 10) #range for the loops in list comprehension below
X = pulp.LpVariable.dicts('X', [(digit, row, col) for digit in nums for row in nums for col in nums], cat = 'Binary') #creating 3D array of binary variables as a dictionary

**Third Step (Constraint Specification):**
We'll need satisfy each of sudoku's rules with a set of constraints:  
I. Ensure that each row, column combination has exactly one value/digit  
II. Ensure that each value/digit occurs in a row exactly once  
III. Ensure that each value/digit occurs in a column exactly once
IV. Ensure that each value/digit occurs in predefined 3x3 squares exactly once  
V. Ensure that hints provided in the original puzzle are coded as constraints  
  
The following lines show how these constraints are written linearly and executed

**Constraint I:**  
We need to ensure that each cell (row j, column k) has exactly one value, i, from 1 through 9. Linearly, this can be written as 



In [27]:
%%latex
\forall j, \forall k: \sum\limits_{i = 1}^9 {X_{ijk}} = 1

<IPython.core.display.Latex object>

In [28]:
for row in nums:
  for col in nums:
    problem += (pulp.lpSum([X[(digit, row, col)] for digit in nums]) == 1)

**Constraint II:**  
We need to ensure that each cell in a row j has exactly one value i from 1 through 9. Linearly, this can be written as 

In [29]:
%%latex
\forall i, \forall k: \sum\limits_{j = 1}^9 {X_{ijk}} = 1

<IPython.core.display.Latex object>

In [30]:
for digit in nums:
  for col in nums:
    problem += (pulp.lpSum([X[(digit, row, col)] for row in nums]) == 1)

**Constraint III:**  
We need to ensure that each cell in a column k has exactly one value i from 1 through 9. Linearly, this can be written as 

In [31]:
%%latex
\forall i, \forall j: \sum\limits_{k = 1}^9 {X_{ijk}} = 1

<IPython.core.display.Latex object>

In [32]:
for digit in nums:
  for row in nums:
      problem += (pulp.lpSum([X[(digit, row, col)] for col in nums]) == 1)

**Constraint IV:**  
We need to ensure that each cell in a 3x3 block has exactly one value i from 1 thorugh 9. Linearly, this can be written as

In [33]:
%%latex 
\forall J, K \in ((1, 2, 3), (4, 5, 6), (7, 8, 9)): \sum\limits_{j \in J} \sum\limits_{k \in K} {X_{ijk}} = 1

<IPython.core.display.Latex object>

In [34]:
nums2 = [range(1, 4), range(4, 7), range(7, 10)]
for digit in nums:
  for row3 in nums2:
    for col3 in nums2:
      problem += (pulp.lpSum([X[(digit, row, col)] for row in row3 for col in col3]) == 1)

**Constraint V:**  
In the original puzzle...  
For each digit $i$ written in a row $j$, column $k$,  
$X_{ijk} = 1$

In [35]:
for row in range(9):
  for col in range(9):
    if puzzle[row][col] != 0:
      problem += (X[(puzzle[row][col], row + 1, col + 1)] == 1)

**Fourth Step: Objective Function**  
Typically, the objective function is written as an expression in the form of a linear expression.  
But in this instance, the focus is on finding parameters that satisfy all of our linear constraints.  
So we can simply state a constant, like $0$, as our objective function.  

In [36]:
problem += 0

**Solving and Displaying the Output**  
Before solving the input puzzle, we should note that there are several types of inputs:
- Type 1: an invalid puzzle is entered. In those scenarios, we'll just return 0
- Type 2: a puzzle with multiple solutions is entered. In this scenario, we'll return the first viable solution that the solver displays
- Type 3: a puzzle with exactly one solution. Here, we'll return the optimal solution

In [37]:
problem.solve()
if problem.solve() == 1:
  sudoku = [digit for row in nums for col in nums for digit in nums  if X[(digit, row, col)].varValue == 1] #list comprehension to store the solution as a 1D array
  output = [sudoku[9 * index: 9 * (index + 1)] for index in range(9)] #convert 1D array to 2D array
else:
  output = 0
output

[[4, 1, 5, 7, 2, 9, 3, 8, 6],
 [3, 9, 2, 1, 8, 6, 7, 5, 4],
 [7, 6, 8, 4, 3, 5, 1, 2, 9],
 [1, 3, 4, 2, 9, 8, 5, 6, 7],
 [5, 2, 9, 6, 7, 4, 8, 3, 1],
 [6, 8, 7, 5, 1, 3, 4, 9, 2],
 [8, 7, 6, 3, 4, 2, 9, 1, 5],
 [2, 4, 3, 9, 5, 1, 6, 7, 8],
 [9, 5, 1, 8, 6, 7, 2, 4, 3]]

**Full Optimization Equation**
Putting together all the objective functions and constraints, the linear program looks like the following:

In [52]:
%%latex
\max \sum\limits_{i=1}^{9} \sum\limits_{j=1}^{9} \sum\limits_{k=1}^{9} 0 \cdot X_{ijk}  \\
s.t. \forall j, \forall k: \sum\limits_{i = 1}^9 {X_{ijk}} = 1 \\
\forall i, \forall k: \sum\limits_{j = 1}^9 {X_{ijk}} = 1 \\
\forall i, \forall j: \sum\limits_{k = 1}^9 {X_{ijk}} = 1 \\
\forall J, K \in ((1, 2, 3), (4, 5, 6), (7, 8, 9)): \sum\limits_{j \in J} \sum\limits_{k \in K} {X_{ijk}} = 1 \\
\exists X_{ijk} \in (0, 1) \forall i, j, k \in (1, 2, 3, ..., 9)^3

<IPython.core.display.Latex object>

**General Function**  
Now that the solve process is clear, we can just create a method to solve puzzles and check test cases

In [53]:
def SudokuSolver(puzzle):

  # Recurring ranges
  nums = range(1, 10)
  nums2 = [range(1, 4), range(4, 7), range(7, 10)]

  # Creating Objective Function
  problem = pulp.LpProblem('Problem', pulp.LpMaximize) #defining a maximization linear program with name 'Problem'
  problem += 0

  # Declaring Variables
  X = pulp.LpVariable.dicts('X', [(digit, row, col) for digit in nums for row in nums for col in nums], cat = 'Binary') #creating 3D array of binary variables as a dictionary
  
  # Constraint 1
  for row in nums:
    for col in nums:
      problem += (pulp.lpSum([X[(digit, row, col)] for digit in nums]) == 1)

  # Constraint 2
  for digit in nums:
    for col in nums:
      problem += (pulp.lpSum([X[(digit, row, col)] for row in nums]) == 1)

  # Constraint 3
  for digit in nums:
    for row in nums:
      problem += (pulp.lpSum([X[(digit, row, col)] for col in nums]) == 1)

  # Constraint 4 
  for digit in nums:
    for row3 in nums2:
      for col3 in nums2:
        problem += (pulp.lpSum([X[(digit, row, col)] for row in row3 for col in col3]) == 1)
  
  # Constraint 5
  for row in range(9):
    for col in range(9):
      if puzzle[row][col] != 0:
        problem += (X[(puzzle[row][col], row + 1, col + 1)] == 1)

  # Solve and Output
  problem.solve()
  if problem.solve() == 1:
    sudoku = [digit for row in nums for col in nums for digit in nums  if X[(digit, row, col)].varValue == 1] #list comprehension to store the solution as a 1D array
    output = [sudoku[9 * index: 9 * (index + 1)] for index in range(9)] #convert 1D array to 2D array
  else:
    output = 0
  return output

**Testing and Accuracy:**  
We'll now try some test cases below to check that the solver is working as intended:  
Test Case 1: Should produce a valid puzzle with unique solution  
Test Case 2: Should produce a valid puzzle with unique solution  
Test Case 3: Assertion error should be thrown since the same value occurs in a row  
Test Case 4: Assertion error should be thrown since the same value occurs in a column  
Test Case 5: Assertion error should be thrown since the same value occurs in a 3x3 block   
Test Case 6: Assertion error should be thrown since constraints 2 - 4 are all violated 


In [56]:
#Test Case 1: Valid Puzzle 1
input1 = [[0, 0, 0, 7, 6, 1, 0, 0, 4], \
          [0, 0, 0, 0, 0, 0, 7, 8, 0], \
          [0, 9, 0, 0, 0, 8, 0, 0, 5], \
          [3, 0, 9, 2, 0, 6, 5, 0, 8], \
          [6, 0, 8, 4, 0, 7, 3, 0, 2], \
          [4, 0, 2, 8, 0, 9, 6, 0, 7], \
          [8, 0, 0, 9, 0, 0, 0, 3, 0], \
          [0, 4, 1, 0, 0, 0, 0, 0, 0], \
          [9, 0, 0, 1, 8, 4, 0, 0, 0]] 
solution1 = [[5, 8, 3, 7, 6, 1, 9, 2, 4], \
             [1, 6, 4, 5, 9, 2, 7, 8, 3], \
             [2, 9, 7, 3, 4, 8, 1, 6, 5], \
             [3, 7, 9, 2, 1, 6, 5, 4, 8], \
             [6, 1, 8, 4, 5, 7, 3, 9, 2], \
             [4, 5, 2, 8, 3, 9, 6, 1, 7], \
             [8, 2, 6, 9, 7, 5, 4, 3, 1], \
             [7, 4, 1, 6, 2, 3, 8, 5, 9], \
             [9, 3, 5, 1, 8, 4, 2, 7, 6]]

#Test Case 2: Valid Puzzle 2
input2 =  [[0, 8, 9, 5, 1, 0, 7, 6, 0], \
           [4, 0, 0, 0, 6, 0, 9, 5, 0], \
           [0, 0, 0, 0, 4, 0, 0, 0, 0], \
           [6, 2, 3, 0, 5, 9, 0, 0, 0], \
           [0, 0, 5, 0, 0, 0, 3, 0, 0], \
           [0, 0, 0, 2, 7, 0, 6, 9, 5], \
           [0, 0, 0, 0, 3, 0, 0, 0, 0], \
           [0, 4, 8, 0, 2, 0, 0, 0, 6], \
           [0, 3, 2, 0, 9, 4, 1, 7, 0]] 
solution2 = [[3, 8, 9, 5, 1, 2, 7, 6, 4], \
             [4, 7, 1, 3, 6, 8, 9, 5, 2], \
             [2, 5, 6, 9, 4, 7, 8, 1, 3], \
             [6, 2, 3, 1, 5, 9, 4, 8, 7], \
             [7, 9, 5, 4, 8, 6, 3, 2, 1], \
             [8, 1, 4, 2, 7, 3, 6, 9, 5], \
             [1, 6, 7, 8, 3, 5, 2, 4, 9], \
             [9, 4, 8, 7, 2, 1, 5, 3, 6], \
             [5, 3, 2, 6, 9, 4, 1, 7, 8]]

#Test Case 3: Violate Constraint 2
input3 = [[2, 2, 2, 2, 2, 2, 2, 2, 2], \
          [0, 0, 0, 0, 0, 0, 0, 0, 0], \
          [0, 0, 0, 0, 0, 0, 0, 0, 0], \
          [0, 0, 0, 0, 0, 0, 0, 0, 0], \
          [0, 0, 0, 0, 0, 0, 0, 0, 0], \
          [0, 0, 0, 0, 0, 0, 0, 0, 0], \
          [0, 0, 0, 0, 0, 0, 0, 0, 0], \
          [0, 0, 0, 0, 0, 0, 0, 0, 0], \
          [0, 0, 0, 0, 0, 0, 0, 0, 0]] 
solution3 = 0

#Test Case 4: Violate Constraint 3
input4 = [[3, 0, 0, 0, 0, 0, 0, 0, 0], \
          [3, 0, 0, 0, 0, 0, 0, 0, 0], \
          [3, 0, 0, 0, 0, 0, 0, 0, 0], \
          [3, 0, 0, 0, 0, 0, 0, 0, 0], \
          [3, 0, 0, 0, 0, 0, 0, 0, 0], \
          [3, 0, 0, 0, 0, 0, 0, 0, 0], \
          [3, 0, 0, 0, 0, 0, 0, 0, 0], \
          [3, 0, 0, 0, 0, 0, 0, 0, 0], \
          [3, 0, 0, 0, 0, 0, 0, 0, 0]] 
solution4 = 0

#Test Case 5: Violate Constraint 4
input5 = [[4, 4, 4, 0, 0, 0, 0, 0, 0], \
          [4, 4, 4, 0, 0, 0, 0, 0, 0], \
          [4, 4, 4, 0, 0, 0, 0, 0, 0], \
          [0, 0, 0, 0, 0, 0, 0, 0, 0], \
          [0, 0, 0, 0, 0, 0, 0, 0, 0], \
          [0, 0, 0, 0, 0, 0, 0, 0, 0], \
          [0, 0, 0, 0, 0, 0, 0, 0, 0], \
          [0, 0, 0, 0, 0, 0, 0, 0, 0], \
          [0, 0, 0, 0, 0, 0, 0, 0, 0]] 
solution5 = 0

#Test Case 6: Violate Constraint 2, 3, and 4
input6 = [[5, 5, 5, 5, 5, 5, 5, 5, 5], \
          [5, 5, 5, 0, 0, 0, 0, 0, 0], \
          [5, 5, 5, 0, 0, 0, 0, 0, 0], \
          [5, 0, 0, 0, 0, 0, 0, 0, 0], \
          [5, 0, 0, 0, 0, 0, 0, 0, 0], \
          [5, 0, 0, 0, 0, 0, 0, 0, 0], \
          [5, 0, 0, 0, 0, 0, 0, 0, 0], \
          [5, 0, 0, 0, 0, 0, 0, 0, 0], \
          [5, 0, 0, 0, 0, 0, 0, 0, 0]] 
solution6 = 0

#Test Case 1: Valid Puzzle 1
assert SudokuSolver(input1) == solution1
print("Test Case 1 Passed: Valid Puzzle 1")

#Test Case 2: Valid Puzzle 2
assert SudokuSolver(input2) == solution2
print("Test Case 2 Passed: Valid Puzzle 2")

#Test Case 3: Violate Constraint 2
assert SudokuSolver(input3) == solution3
print("Test Case 3 Passed: Violate Constraint 2")

#Test Case 4: Violate Constraint 3
assert SudokuSolver(input4) == solution4
print("Test Case 4 Passed: Violate Constraint 3")

#Test Case 5: Violate Constraint 4
assert SudokuSolver(input5) == solution5
print("Test Case 5 Passed: Violate Constraint 4")

#Test Case 6: Violate Constraint 2, 3, and 4  
assert SudokuSolver(input6) == solution6
print("Test Case 6 Passed: Violate Constraint 2, 3, and 4")

print("\n PASSED ALL TEST CASES!")

Test Case 1 Passed: Valid Puzzle 1
Test Case 2 Passed: Valid Puzzle 2
Test Case 3 Passed: Violate Constraint 2
Test Case 4 Passed: Violate Constraint 3
Test Case 5 Passed: Violate Constraint 4
Test Case 6 Passed: Violate Constraint 2, 3, and 4

 PASSED ALL TEST CASES!


**Run Times**  
When each of the constraints were previously stated, you may have noticed that 2-3 for loops were utilized for each constraint.  
This may have cut down the number of manually written constraints from 324 linear equations to 4 generalized conditions, but...  
the current setup could potentially strangle run time at scale (for sudoku solvers in number bases higher than 10) since each constraint would run in O(n2) or O(n3) time  

So just to check the run time of our puzzles for standard base 10 sudoku puzzles, we'll use python's timeit feature below on our first two test cases





In [57]:
%timeit SudokuSolver(input1)
%timeit SudokuSolver(input2)

10 loops, best of 3: 100 ms per loop
10 loops, best of 3: 96.3 ms per loop


**Result**  
Based on these initial inputs, our solver runs correctly in approximately 0.1 seconds.  
Hope you've enjoyed this tutorial!